In [1]:
import pandas_datareader.data as web #to collect data
import datetime as dt #to specify start and end dates

# import yfinance as yf

import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

from patsy import dmatrices
from tqdm.notebook import tqdm
tqdm.pandas()

## Data reading and melting

In [2]:
import_folder_path = r"..\..\[IN USE] Rookie Directors\[1] Director Level\director_wrangle_output"
output_folder_path = "car_output1"
supporting_folder_path = "supporting_datafiles"

In [3]:
data = pd.read_csv(rf"{supporting_folder_path}\Adjusted Clos_collated.csv").drop("Unnamed: 0", axis = 1)

In [4]:
data

,AsOnDate,20 Microns Ltd.,20Th Century Finance Corpn. Ltd. [Merged],360 One Wam Ltd.,3I Infotech Ltd.,3M India Ltd.,3P Land Holdings Ltd.,3Rd Rock Multimedia Ltd.,5Paisa Capital Ltd.,63 Moons Technologies Ltd.,...,3898,3899,3900,3901,3902,3903,3904,3905,3906,3907
0,2004-01-01,NaN,NaN,NaN,NaN,515.00,NaN,NaN,NaN,NaN,...,66.83,NaN,15.25,NaN,NaN,NaN,59.85,25.79,NaN,NaN
1,2004-01-02,NaN,NaN,NaN,NaN,530.50,3.20,NaN,NaN,NaN,...,65.88,NaN,17.35,NaN,NaN,NaN,66.50,26.74,NaN,NaN
2,2004-01-05,NaN,NaN,NaN,NaN,511.75,3.42,NaN,NaN,NaN,...,61.76,NaN,16.30,NaN,NaN,NaN,62.70,25.95,NaN,NaN
3,2004-01-06,NaN,NaN,NaN,NaN,495.00,NaN,NaN,NaN,NaN,...,61.94,NaN,15.70,NaN,NaN,NaN,59.50,25.74,NaN,NaN
4,2004-01-07,NaN,NaN,NaN,NaN,490.40,2.76,NaN,NaN,NaN,...,60.33,NaN,16.00,NaN,NaN,NaN,57.75,25.15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,2024-03-21,144.70,12.5,673.40,41.60,30049.25,30.80,64.6,487.45,407.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5026,2024-03-22,145.05,12.5,665.85,41.85,30728.40,30.40,64.6,487.85,402.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5027,2024-03-26,143.00,12.5,650.30,39.20,30487.15,28.90,64.6,476.05,393.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5028,2024-03-27,143.00,12.5,667.95,39.40,31475.10,28.40,64.6,490.30,381.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dataLong = data.melt( id_vars = "AsOnDate", value_vars = data.columns[1:3908]).rename({"value":"ACP", "variable":"CompanyName"}, axis = 1).drop_duplicates().reset_index(drop = True)
dataLong = dataLong.loc[~( (dataLong.duplicated(subset = ["CompanyName", "AsOnDate"], keep = False)) & (dataLong["ACP"].isnull())) ]
dataLong = dataLong.loc[~ dataLong.duplicated(subset = ["CompanyName", "AsOnDate"], keep = False)].drop_duplicates().reset_index(drop = True)

In [6]:
dataLong

,AsOnDate,CompanyName,ACP
0,2004-01-01,20 Microns Ltd.,NaN
1,2004-01-02,20 Microns Ltd.,NaN
2,2004-01-05,20 Microns Ltd.,NaN
3,2004-01-06,20 Microns Ltd.,NaN
4,2004-01-07,20 Microns Ltd.,NaN
...,...,...,...
19640110,2024-03-21,3907,NaN
19640111,2024-03-22,3907,NaN
19640112,2024-03-26,3907,NaN
19640113,2024-03-27,3907,NaN


## Data Cleaning

### Data Snipping from either ends

In [7]:
def dataSnip(frame):
    
    first_valid_idx = frame["ACP"].first_valid_index()
    
    if first_valid_idx is not None:
        frame = frame.loc[first_valid_idx:]
        
    else:
        frame = frame.iloc[0:0]

    last_valid_idx = frame["ACP"].last_valid_index()
    
    if last_valid_idx is not None:
        frame = frame.loc[:last_valid_idx]
        
    else:
        frame = frame
        
    return frame

In [8]:
dataLong2 = dataLong.groupby(by="CompanyName").progress_apply(dataSnip).reset_index(drop=True)
dataLong2

  0%|          | 0/3907 [00:00<?, ?it/s]

,AsOnDate,CompanyName,ACP
0,2008-10-06,20 Microns Ltd.,16.82
1,2008-10-07,20 Microns Ltd.,15.05
2,2008-10-08,20 Microns Ltd.,13.25
3,2008-10-10,20 Microns Ltd.,11.60
4,2008-10-13,20 Microns Ltd.,12.32
...,...,...,...
13465088,2024-03-21,Zylog Systems Ltd.,0.35
13465089,2024-03-22,Zylog Systems Ltd.,0.35
13465090,2024-03-26,Zylog Systems Ltd.,0.35
13465091,2024-03-27,Zylog Systems Ltd.,0.35


### Inter series NaN values filled as the previous value

In [9]:
def dataForwardFill(frame):

    frame["ACP"] = frame["ACP"].ffill()
        
    return frame

In [10]:
dataLong3 = dataLong2.groupby(by="CompanyName").apply(dataForwardFill).reset_index(drop=True)
dataLong3

,AsOnDate,CompanyName,ACP
0,2008-10-06,20 Microns Ltd.,16.82
1,2008-10-07,20 Microns Ltd.,15.05
2,2008-10-08,20 Microns Ltd.,13.25
3,2008-10-10,20 Microns Ltd.,11.60
4,2008-10-13,20 Microns Ltd.,12.32
...,...,...,...
13465088,2024-03-21,Zylog Systems Ltd.,0.35
13465089,2024-03-22,Zylog Systems Ltd.,0.35
13465090,2024-03-26,Zylog Systems Ltd.,0.35
13465091,2024-03-27,Zylog Systems Ltd.,0.35


### Result

In [11]:
dataLong3.ACP.isnull().value_counts()

ACP
False    13465093
Name: count, dtype: int64

In [12]:
closingPrice = dataLong3.copy()

## Simple Returns

In [13]:
def pct_change(frame):
    frame = frame.sort_values(by = ["AsOnDate"])
    frame["pct"] = frame["ACP"].pct_change(fill_method = None)
    return frame

In [14]:
simpleReturn = closingPrice.groupby("CompanyName").apply(pct_change).reset_index(drop = True)

In [15]:
simpleReturn["AsOnDate"] = pd.to_datetime(simpleReturn["AsOnDate"], format = "%Y-%m-%d")
simpleReturn = simpleReturn.drop_duplicates().reset_index(drop = True)
simpleReturn

,AsOnDate,CompanyName,ACP,pct
0,2008-10-06,20 Microns Ltd.,16.82,NaN
1,2008-10-07,20 Microns Ltd.,15.05,-0.105232
2,2008-10-08,20 Microns Ltd.,13.25,-0.119601
3,2008-10-10,20 Microns Ltd.,11.60,-0.124528
4,2008-10-13,20 Microns Ltd.,12.32,0.062069
...,...,...,...,...
13465088,2024-03-21,Zylog Systems Ltd.,0.35,0.000000
13465089,2024-03-22,Zylog Systems Ltd.,0.35,0.000000
13465090,2024-03-26,Zylog Systems Ltd.,0.35,0.000000
13465091,2024-03-27,Zylog Systems Ltd.,0.35,0.000000


## FF 3 constants

In [16]:
ff3Const = pd.read_csv(rf"{supporting_folder_path}\2024-03_FourFactors_and_Market_Returns_Daily_SurvivorshipBiasAdjusted.csv")

In [17]:
ff3Const["date"] = pd.to_datetime(ff3Const["date"], format = "%d-%m-%Y")

In [18]:
ff3Const = ff3Const.rename({"date":"AsOnDate"}, axis = 1).drop_duplicates().reset_index(drop = True)
ff3Const["RMRF"] = ff3Const["MF"] - ff3Const["RF"]
ff3Const.columns[[0,1,2,3,4,5,6]]
orderedCol = ff3Const.columns[[0, 5, 6, 4, 1, 2]]
ff3ConstOrdered = ff3Const[orderedCol]
ff3ConstOrdered

,AsOnDate,RF,RMRF,MF,SMB,HML
0,1993-10-01,NaN,NaN,NaN,1.414154,1.182552
1,1993-10-04,0.022014,-0.954330,-0.932316,0.472301,0.371959
2,1993-10-05,0.022014,-0.315512,-0.293498,0.046619,0.839734
3,1993-10-06,0.022014,-0.352836,-0.330823,-0.085561,-1.492747
4,1993-10-07,0.022014,0.360946,0.382959,-0.286668,0.135259
...,...,...,...,...,...,...
7563,2024-03-21,0.018215,1.441807,1.460021,0.572866,1.270303
7564,2024-03-22,0.018215,0.593324,0.611538,0.752002,0.124527
7565,2024-03-26,0.072878,-0.044514,0.028364,-1.091597,0.440605
7566,2024-03-27,0.018215,0.326702,0.344916,-0.165072,-0.188505


In [19]:
# df = ff3ConstOrdered.copy()
# df["Year"] = float(df["AsOnDate"].year)
# df.groupby(["Year"]).apply(

## Merged and Final Data set

In [20]:
dataMerged1 = simpleReturn.merge(ff3ConstOrdered, on = "AsOnDate", how = "inner").sort_values(["CompanyName", "AsOnDate"]).set_index(["CompanyName", "AsOnDate"]).reset_index().drop_duplicates().reset_index(drop = True)

In [21]:
dataMerged1

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853
...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505


## Company Keys and Merging with data above

In [22]:
companyKeys = pd.read_excel(rf"{supporting_folder_path}\Prowess Code_NSE Symbol.xlsx").rename({"Company Name":"CompanyName", "Prowess company code":"ProwessCode", "NSE symbol":"Symbol"}, axis = 1 )

In [23]:
companyKeys

,CompanyName,ProwessCode,Symbol
0,'K' Steamship Agencies Pvt. Ltd.,3,NaN
1,'X'Clusive Business Centre Pvt. Ltd.,307865,NaN
2,1 To 1 Help.Net Pvt. Ltd.,591675,NaN
3,10C India Internet Pvt. Ltd.,556976,NaN
4,10I Commerce Services Pvt. Ltd.,560502,NaN
...,...,...,...
55307,Zylog Systems Ltd.,275793,ZYLOG
55308,Zyma Laboratories Ltd. [Merged],275794,NaN
55309,Zyphar'S Pharmaceutics Pvt. Ltd.,565342,NaN
55310,Zytel Agencies Ltd.,275795,NaN


In [24]:
dataMerged2 = companyKeys.merge(dataMerged1, on = "CompanyName", how = "right").sort_values( by = ["CompanyName", "AsOnDate"])

In [25]:
# The above are not in the director dataset, so can be ignored
dataMerged2

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853
...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505


In [26]:
dataMerged2["AsOnDate"] = pd.to_datetime(dataMerged2["AsOnDate"], format = "%Y-%m-%d")

## Director Appointment Dates

In [27]:
dir = pd.read_pickle(rf"{import_folder_path}\Main_Director_COMPLETE.pkl")

In [28]:
dirAppointment = dir.loc[:, ["Symbol", "Appointment Date"]].copy().sort_values( by = ["Symbol", "Appointment Date"]).drop_duplicates().reset_index(drop = True)
dirAppointment

,Symbol,Appointment Date
0,20MICRONS,1988-03-29
1,20MICRONS,1998-07-02
2,20MICRONS,2000-04-10
3,20MICRONS,2001-01-29
4,20MICRONS,2001-02-27
...,...,...
39086,ZYLOG,2014-11-19
39087,ZYLOG,2015-08-14
39088,ZYLOG,2015-11-25
39089,ZYLOG,2016-11-23


### Merging Appointment Dates within dataMerged

In [29]:
dirMerged = dataMerged2.merge(dirAppointment, left_on = ["Symbol", "AsOnDate"], right_on = ["Symbol", "Appointment Date"], how = "left")

In [30]:
dirMerged

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,Appointment Date
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaT
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaT
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaT
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaT
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaT
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaT
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaT
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaT


# Organic Functions

In [31]:
# Just Trying
# FIT OLS, with R style formulas
# res = smf.ols("pct - RF ~ RMRF + SMB + HML", data = dataMerged).fit()

In [32]:
# res.summary()

In [33]:
dirMerged.CompanyName.nunique()

3721

## 120 Day Event Study

### 120 day window OLS

In [34]:
# pre-event 120 days
# ignoring companies with <130 data points in full.

def OLS120(frame):
    frame = frame.reset_index(drop = True)
    results = []
    if len(frame) > 151:
        
    #outputFrame = pd.DataFrame( columns = ["Appointment Date", "const", "RMRF", "SMB", "HML", "OLS120_r_squared", "OLS120_adjusted_r_squared", "OLS120_f_p_value"])
        
        for date in range(len(frame["Appointment Date"])) :
            if not pd.isnull(frame.iloc[date]["Appointment Date"]):
                if date >= 151:
                    ols = frame.iloc[ date - 151 : date - 31]
                    exog_vars = ["RMRF", "SMB", "HML"]
                    endog = ols.pct - ols.RF
                    exog = sm.add_constant(ols[exog_vars])
                    rols = sm.OLS(endog, exog)
                    res = rols.fit()
    
                    outputFrame1 = res.params.to_frame().T
                    outputFrame1["OLS120_r_squared"] = res.rsquared
                    outputFrame1["OLS120_adjusted_r_squared"] = res.rsquared_adj
                    outputFrame1["OLS120_f_p_value"] = res.f_pvalue
                    outputFrame1["Appointment Date"] = frame.iloc[date]["Appointment Date"]
    
                    if not outputFrame1.isnull().all().all():  # Ensure it's not all NaNs
                        results.append(outputFrame1)
    
        return pd.concat(results, ignore_index=True) if results else pd.DataFrame()

In [35]:
ols120Param = dirMerged.groupby(by = ["CompanyName"]).progress_apply(OLS120).reset_index()
ols120Param = ols120Param.rename({"const":"OLS120_intercept", "RMRF":"OLS120_RMRF", "SMB":"OLS120_SMB", "HML":"OLS120_HML"}, axis = 1).drop("level_1", axis = 1)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [36]:
ols120Param

,CompanyName,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,Appointment Date
0,20 Microns Ltd.,-0.013787,0.004889,0.011979,0.005721,0.127494,0.104929,1.195618e-03,2009-08-27
1,20 Microns Ltd.,-0.027096,0.010449,0.015732,-0.002440,0.176213,0.154908,4.937782e-05,2011-04-29
2,20 Microns Ltd.,-0.035800,0.010683,0.006169,0.001164,0.121640,0.098924,1.725805e-03,2014-08-06
3,20 Microns Ltd.,-0.024574,0.014913,0.012525,0.005669,0.359471,0.342905,3.165739e-11,2017-05-04
4,20 Microns Ltd.,-0.025666,0.014600,0.013163,0.003634,0.308224,0.290333,2.555217e-09,2019-05-28
...,...,...,...,...,...,...,...,...,...
20525,Zylog Systems Ltd.,-0.039007,0.014935,0.013479,0.010621,0.124132,0.101480,1.476820e-03,2014-02-14
20526,Zylog Systems Ltd.,-0.039050,0.003156,0.011326,0.012391,0.156904,0.135100,1.795678e-04,2014-11-19
20527,Zylog Systems Ltd.,-0.034650,0.004784,-0.007787,0.006894,0.045213,0.020521,1.453695e-01,2015-08-14
20528,Zylog Systems Ltd.,-0.028118,0.000909,-0.002034,0.008051,0.057325,0.032945,7.591845e-02,2016-11-23


In [37]:
ols120 = dirMerged.merge(ols120Param, left_on = ["CompanyName", "AsOnDate"], right_on = ["CompanyName", "Appointment Date"], how = "left").drop(["Appointment Date_x", "Appointment Date_y"], axis = 1)

In [38]:
ols120.to_pickle(rf"{output_folder_path}\ols120_2.pkl")

In [39]:
del ols120Param
del ols120

## 150 Day Event Study

### 150 day window OLS

In [40]:
# pre-event 150 days
# ignoring companies with <130 data points in full.

def OLS150(frame):
    frame = frame.reset_index(drop = True)
    results = []
    if len(frame) > 181:
    
        #outputFrame = pd.DataFrame( columns = ["Appointment Date", "const", "RMRF", "SMB", "HML", "OLS150_r_squared", "OLS150_adjusted_r_squared", "OLS150_f_p_value"])
        
        for date in range(len(frame["Appointment Date"])) :
            if not pd.isnull(frame.iloc[date]["Appointment Date"]):
                if date >= 181:
                    ols = frame.iloc[ date - 181 : date - 31]
                    exog_vars = ["RMRF", "SMB", "HML"]
                    endog = ols.pct - ols.RF
                    exog = sm.add_constant(ols[exog_vars])
                    rols = sm.OLS(endog, exog)
                    res = rols.fit()
    
                    outputFrame1 = res.params.to_frame().T
                    outputFrame1["OLS150_r_squared"] = res.rsquared
                    outputFrame1["OLS150_adjusted_r_squared"] = res.rsquared_adj
                    outputFrame1["OLS150_f_p_value"] = res.f_pvalue
                    outputFrame1["Appointment Date"] = frame.iloc[date]["Appointment Date"]
    
                    if not outputFrame1.isnull().all().all():  # Ensure it's not all NaNs
                        results.append(outputFrame1)
    
        return pd.concat(results, ignore_index=True) if results else pd.DataFrame()

In [41]:
ols150Param = dirMerged.groupby(by = ["CompanyName"]).progress_apply(OLS150).reset_index()
ols150Param = ols150Param.rename({"const":"OLS150_intercept", "RMRF":"OLS150_RMRF", "SMB":"OLS150_SMB", "HML":"OLS150_HML"}, axis = 1).drop("level_1", axis = 1)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [42]:
ols150Param

,CompanyName,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,Appointment Date
0,20 Microns Ltd.,-0.017060,0.006434,0.011588,0.008780,0.163067,0.145870,9.189620e-06,2009-08-27
1,20 Microns Ltd.,-0.025992,0.008764,0.012305,0.002310,0.158002,0.140700,1.406897e-05,2011-04-29
2,20 Microns Ltd.,-0.035619,0.008433,0.006457,0.001327,0.084912,0.066109,4.638373e-03,2014-08-06
3,20 Microns Ltd.,-0.023298,0.017193,0.013819,0.001278,0.268181,0.253143,6.453930e-10,2017-05-04
4,20 Microns Ltd.,-0.025053,0.015349,0.011943,0.002961,0.361151,0.348024,3.667411e-14,2019-05-28
...,...,...,...,...,...,...,...,...,...
20401,Zylog Systems Ltd.,-0.042799,0.017896,0.017156,0.007030,0.135180,0.117410,9.226997e-05,2014-02-14
20402,Zylog Systems Ltd.,-0.039048,0.005378,0.008079,0.012985,0.146228,0.128685,3.741366e-05,2014-11-19
20403,Zylog Systems Ltd.,-0.034041,0.006057,-0.003198,0.008756,0.079905,0.060999,6.729009e-03,2015-08-14
20404,Zylog Systems Ltd.,-0.029026,0.001455,-0.002229,0.005384,0.035742,0.015929,1.490415e-01,2016-11-23


In [43]:
ols150 = dirMerged.merge(ols150Param, left_on = ["CompanyName", "AsOnDate"], right_on = ["CompanyName", "Appointment Date"], how = "left").drop(["Appointment Date_x", "Appointment Date_y"], axis = 1)

In [44]:
ols150

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
ols150.to_pickle(rf"{output_folder_path}\ols150_2.pkl")

In [46]:
del ols150Param
del ols150

## 180 Day Event Study

### 180 day window OLS

In [47]:
# pre-event 180 days
# ignoring companies with <130 data points in full.

def OLS180(frame):
    frame = frame.reset_index(drop = True)
    results = []
    if len(frame) > 211:
    
        #outputFrame = pd.DataFrame( columns = ["Appointment Date", "const", "RMRF", "SMB", "HML", "OLS180_r_squared", "OLS180_adjusted_r_squared", "OLS180_f_p_value"])
        
        for date in range(len(frame["Appointment Date"])) :
            if not pd.isnull(frame.iloc[date]["Appointment Date"]):
                if date >= 211:
                    ols = frame.iloc[ date - 211 : date - 31]
                    exog_vars = ["RMRF", "SMB", "HML"]
                    endog = ols.pct - ols.RF
                    exog = sm.add_constant(ols[exog_vars])
                    rols = sm.OLS(endog, exog)
                    res = rols.fit()
    
                    outputFrame1 = res.params.to_frame().T
                    outputFrame1["OLS180_r_squared"] = res.rsquared
                    outputFrame1["OLS180_adjusted_r_squared"] = res.rsquared_adj
                    outputFrame1["OLS180_f_p_value"] = res.f_pvalue
                    outputFrame1["Appointment Date"] = frame.iloc[date]["Appointment Date"]
    
                    if not outputFrame1.isnull().all().all():  # Ensure it's not all NaNs
                        results.append(outputFrame1)
    
        return pd.concat(results, ignore_index=True) if results else pd.DataFrame()

In [48]:
ols180Param = dirMerged.groupby(by = ["CompanyName"]).progress_apply(OLS180).reset_index()
ols180Param = ols180Param.rename({"const":"OLS180_intercept", "RMRF":"OLS180_RMRF", "SMB":"OLS180_SMB", "HML":"OLS180_HML"}, axis = 1).drop("level_1", axis = 1)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [49]:
ols180Param

,CompanyName,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,Appointment Date
0,20 Microns Ltd.,-0.019523,0.008528,0.010563,0.007012,0.162659,0.148386,7.238692e-07,2009-08-27
1,20 Microns Ltd.,-0.023910,0.008059,0.012570,0.004786,0.128802,0.113952,2.125338e-05,2011-04-29
2,20 Microns Ltd.,-0.035592,0.008052,0.006294,0.002610,0.076637,0.060898,2.808170e-03,2014-08-06
3,20 Microns Ltd.,-0.023958,0.016109,0.010135,0.001335,0.219310,0.206003,1.752317e-09,2017-05-04
4,20 Microns Ltd.,-0.023995,0.016299,0.012980,0.002108,0.329361,0.317930,3.320964e-15,2019-05-28
...,...,...,...,...,...,...,...,...,...
20158,Zylog Systems Ltd.,-0.040186,0.018688,0.016930,0.006755,0.133560,0.118791,1.334846e-05,2014-02-14
20159,Zylog Systems Ltd.,-0.038065,0.004418,0.009466,0.014045,0.158676,0.144335,1.086363e-06,2014-11-19
20160,Zylog Systems Ltd.,-0.033714,0.005668,-0.003566,0.012798,0.106314,0.091081,1.833697e-04,2015-08-14
20161,Zylog Systems Ltd.,-0.030886,0.002208,-0.003777,0.004163,0.031962,0.015461,1.253331e-01,2016-11-23


In [50]:
ols180 = dirMerged.merge(ols180Param, left_on = ["CompanyName", "AsOnDate"], right_on = ["CompanyName", "Appointment Date"], how = "left").drop(["Appointment Date_x", "Appointment Date_y"], axis = 1)

In [51]:
ols180

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
ols180.to_pickle(rf"{output_folder_path}\ols180_2.pkl")

In [53]:
del ols180Param
del ols180

## 210 Day Event Study

### 210 day window OLS

In [54]:
# pre-event 210 days
# ignoring companies with <240 data points in full.

def OLS210(frame):
    frame = frame.reset_index(drop = True)
    results = []
    if len(frame) > 241:
    
        #outputFrame = pd.DataFrame( columns = ["Appointment Date", "const", "RMRF", "SMB", "HML", "OLS210_r_squared", "OLS210_adjusted_r_squared", "OLS210_f_p_value"])
        
        for date in range(len(frame["Appointment Date"])) :
            if not pd.isnull(frame.iloc[date]["Appointment Date"]):
                if date >= 241:
                    ols = frame.iloc[ date - 241 : date - 31]
                    exog_vars = ["RMRF", "SMB", "HML"]
                    endog = ols.pct - ols.RF
                    exog = sm.add_constant(ols[exog_vars])
                    rols = sm.OLS(endog, exog)
                    res = rols.fit()
    
                    outputFrame1 = res.params.to_frame().T
                    outputFrame1["OLS210_r_squared"] = res.rsquared
                    outputFrame1["OLS210_adjusted_r_squared"] = res.rsquared_adj
                    outputFrame1["OLS210_f_p_value"] = res.f_pvalue
                    outputFrame1["Appointment Date"] = frame.iloc[date]["Appointment Date"]
    
                    if not outputFrame1.isnull().all().all():  # Ensure it's not all NaNs
                        results.append(outputFrame1)
    
        return pd.concat(results, ignore_index=True) if results else pd.DataFrame()

In [55]:
ols210Param = dirMerged.groupby(by = ["CompanyName"]).progress_apply(OLS210).reset_index()
ols210Param = ols210Param.rename({"const":"OLS210_intercept", "RMRF":"OLS210_RMRF", "SMB":"OLS210_SMB", "HML":"OLS210_HML"}, axis = 1).drop("level_1", axis = 1)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [56]:
ols210Param

,CompanyName,OLS210_intercept,OLS210_RMRF,OLS210_SMB,OLS210_HML,OLS210_r_squared,OLS210_adjusted_r_squared,OLS210_f_p_value,Appointment Date
0,20 Microns Ltd.,-0.022830,0.009052,0.012831,0.002836,0.131758,0.119114,2.058205e-06,2011-04-29
1,20 Microns Ltd.,-0.036365,0.006364,0.006151,0.003198,0.065439,0.051829,2.939000e-03,2014-08-06
2,20 Microns Ltd.,-0.024819,0.014223,0.010697,0.003037,0.206107,0.194546,2.516556e-10,2017-05-04
3,20 Microns Ltd.,-0.024276,0.015716,0.012604,0.001488,0.334962,0.325277,3.803786e-18,2019-05-28
4,20 Microns Ltd.,-0.023586,0.014998,0.013664,0.007173,0.313242,0.303240,1.008558e-16,2023-05-16
...,...,...,...,...,...,...,...,...,...
20041,Zylog Systems Ltd.,-0.038926,0.017004,0.015859,0.008312,0.132494,0.119861,1.890948e-06,2014-02-14
20042,Zylog Systems Ltd.,-0.038244,0.003994,0.009574,0.014231,0.158091,0.145830,9.396380e-08,2014-11-19
20043,Zylog Systems Ltd.,-0.034644,0.006125,0.000900,0.012578,0.116788,0.103926,1.131949e-05,2015-08-14
20044,Zylog Systems Ltd.,-0.027645,0.001205,-0.002831,0.004653,0.025289,0.011094,1.518053e-01,2016-11-23


In [57]:
ols210 = dirMerged.merge(ols210Param, left_on = ["CompanyName", "AsOnDate"], right_on = ["CompanyName", "Appointment Date"], how = "left").drop(["Appointment Date_x", "Appointment Date_y"], axis = 1)

In [58]:
ols210

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS210_intercept,OLS210_RMRF,OLS210_SMB,OLS210_HML,OLS210_r_squared,OLS210_adjusted_r_squared,OLS210_f_p_value
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
ols210.to_pickle(rf"{output_folder_path}\ols210_2.pkl")

In [60]:
ols210

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS210_intercept,OLS210_RMRF,OLS210_SMB,OLS210_HML,OLS210_r_squared,OLS210_adjusted_r_squared,OLS210_f_p_value
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,NaN,NaN
